<script>
    function findAncestor (el, name) {
        while ((el = el.parentElement) && el.nodeName.toLowerCase() !== name);
        return el;
    }
    function colorAll(el, textColor) {
        el.style.color = textColor;
        Array.from(el.children).forEach((e) => {colorAll(e, textColor);});
    }
    function setBackgroundImage(src, textColor) {
        var section = findAncestor(document.currentScript, 'section');
        if (section) {
            section.setAttribute('data-background-image', src);
			if (textColor) colorAll(section, textColor);
        }
    }
</script>

<style>
h1 {
  border: 1.5px solid #333;
  padding: 8px 12px;
  background-image: linear-gradient(#2774AE,#ebf8e1, #FFD100);
  position: static;
}
</style>

<h1 style='color:white'> Statistics 21 <br/> Python & Other Technologies for Data Science </h1>

<h3 style='color:white'>Vivian Lew, PhD - Monday, Week 10</h3>

<script>
    setBackgroundImage('Window1.jpg');
</script>

# Practicing SQL in Python

SQL Examples from SQL Cookbook 2nd edition

Access for UCLA Students:

https://www.oreilly.com/library/view/sql-cookbook-2nd/9781492077435/?ar

## Please Do not try to set up your own SQL Server

There is a big difference between setting up a SQL server and learning how to write a few queries.

We will learn how to write a few queries.

We will not learn how to set up a server and design a database. Most likely, if you go into the workforce and they list SQL (structured query language) as a requirement, they need someone who can write queries. 

The person who sets up, designs, and maintains the server will be a database administrator (typically someone from a CS background)

## Going Beyond and Learning More

Recommended exercises: <https://www.w3resource.com/sql-exercises/>

Another place to just practice SQL queries: <http://sqlfiddle.com>

This tutorial is taken directly from https://docs.python.org/3/library/sqlite3.html

### What is SQLite?

SQLite is a C library 

It is a lightweight disk-based database that doesn’t require a separate physical server or server process.

It allows accessing of the database using a variant of SQL. The syntax is based in SQL and if you learn basic SQL using SQLite you should be able to make the leap to full blown SQL.

The databases in SQLite are stored in a single file, that's the most obvious difference today.

So it is best used on small datasets and it is suitable for this class (max size is 140 terabytes TB)

In [1]:
import pandas as pd
import sqlite3
con = sqlite3.connect("tutorial.db")

If tutorial.db does not exist, we've just created it.

If tutorial.db does exist, we connected to it and can use it now.

Next step, define a cursor

In [2]:
cur = con.cursor()

a cursor is a tool that helps you navigate through and it allows you to process individual row (record) returned by a query.

### Creating a table

In [3]:
# you may need to run this if you already have run
# the tutorial before
# cur.execute("DROP TABLE movie")

In [4]:
cur.execute("CREATE TABLE movie(title, year, score)")

Tables are database objects that contain all the data in a database.

Notice we are just listing the variable names in the declaration of columns, specifying the data type is optional (this is not typically true in SQL)

### On execute("SQL string")

The execute method in sqlite3's cursor object is used to execute a SQL command. 

It is how we interact with the database and perform operations such as creating a table, inserting data into a table, updating data, deleting data, and querying data.

### Verify that we did create a table

sqlite_master is an internal table that is present in all SQLite databases. The content of this table describes the database's schema (structure of the database)

The fetchone() method is used to fetch the next row. 

It returns either a single record or a None if no rows are available.

In [5]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie2',)

In [6]:
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
res.fetchone() is None 

True

### Adding data to the table

Currently, the table is empty, but we can populate it.  First, the old and slow way:

In [7]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

This is known as a "transaction".  In the world of databases, this is a unit of work, typically, transaction represents any change in a database.

### Committing the transaction

The INSERT statement opened the transaction 

A transaction must be committed before changes are saved in the database (

Calling con.commit() on the connection object to commit the transaction:

In [8]:
con.commit()

### Verification

We can check to see that the database was modified.  The fetchall() method will return all of the rows.

In [9]:
res = cur.execute("SELECT score FROM movie")
res.fetchall()

[(8.2,), (7.5,)]

### Insert

Here, we have data existing in a separate object

executemany() will repeatedly execute the SQL statement for every parameterized item:

In [10]:
data = [
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()  # Remember to commit the transaction after executing INSERT.

## More verification and a use

Again, we verify, but we also see how to being using the database in a Python script

In [11]:
res = cur.execute("SELECT score FROM movie")
res.fetchall()

[(8.2,), (7.5,), (7.5,), (7.9,), (8.0,)]

In [12]:
for row in cur.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)


(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


### Clean-up

close() will close the database connection and we should (not required, but advised) close the cursor object.

In [13]:
cur.close()
con.close()

We can reconnect and re-use our created data.

In [14]:
new_con = sqlite3.connect("tutorial.db")
new_cur = new_con.cursor()
res = new_cur.execute("SELECT title, year FROM movie ORDER BY score DESC")
title, year = res.fetchone()
print(f'The highest scoring Monty Python movie is {title!r}, released in {year}')

The highest scoring Monty Python movie is 'Monty Python and the Holy Grail', released in 1975


In [15]:
new_cur.close()
new_con.close()

## Notes on the Basics

SQL is generally not case sensitive. Convention, however, puts SQL commands in ALL-CAPS and then leaves column and variable names in the same case as they appear in the table.

semi-colons are not required to terminate SQL queries, but their usage is recommended.

### `SELECT`

`SELECT` is used to select variables from a given table. To select all columns, use `*`

### `FROM`

`FROM` specifies which table to select from.

Once you create the database engine with `sqlite3`, we can begin executing SQL queries by establishing a connection with the database.

In [16]:
## how about pandas

tweets = pd.read_csv("http://www.stat.ucla.edu/~vlew/datasets/tweets_01-08-2021.csv")
tweets.set_index('id')

,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
id,,,,,,,,
98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
...,...,...,...,...,...,...,...,...
1319485303363571714,RT @RandPaul: I don’t know why @JoeBiden think...,t,f,Twitter for iPhone,0,20683,2020-10-23 03:46:25,f
1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,t,f,Twitter for iPhone,0,9869,2020-10-23 03:42:05,f
1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Deba...,t,f,Twitter for iPhone,0,8197,2020-10-23 01:03:58,f


In [17]:
table_name = 'Tweets'

pd_con = sqlite3.connect('tweets.db')
query = f'CREATE TABLE IF NOT EXISTS {table_name} \
(text, isRetweet, isDeleted, device, favorites, \
retweets, date, isFlagged)'
pd_con.execute(query)
tweets.to_sql(table_name, pd_con, if_exists='replace', index = False)
pd_con.commit()
pd_con.close()

## Did it work?

In [18]:
new_pd_con = sqlite3.connect("tweets.db")
res = new_pd_con.execute("SELECT * FROM Tweets ORDER BY favorites DESC")
res.fetchone()

(1311892190680014849,
 'Tonight, @FLOTUS and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!',
 'f',
 'f',
 'Twitter for iPhone',
 1869706,
 408866,
 '2020-10-02 04:54:06',
 'f')

In [19]:
new_pd_con.close()

Using the execute() method on a connection actually creates a new cursor, executes the command, and then discards the cursor.  This is a handy shortcut for simple examples but not when you have complex SQL commands.

## Viewing all of the fields in SQL

In [20]:
new_con = sqlite3.connect("tweets.db")
new_cur = new_con.cursor()
new_cur.execute('PRAGMA table_info(Tweets);')
table_info = new_cur.fetchall()

for column in table_info:
    print(column)

new_cur.close()
new_con.close()

(0, 'id', 'INTEGER', 0, None, 0)
(1, 'text', 'TEXT', 0, None, 0)
(2, 'isRetweet', 'TEXT', 0, None, 0)
(3, 'isDeleted', 'TEXT', 0, None, 0)
(4, 'device', 'TEXT', 0, None, 0)
(5, 'favorites', 'INTEGER', 0, None, 0)
(6, 'retweets', 'INTEGER', 0, None, 0)
(7, 'date', 'TEXT', 0, None, 0)
(8, 'isFlagged', 'TEXT', 0, None, 0)


### The inflexibility of SQLite

Suppose now we wanted to identify 'id' as the primary key.

We need to remake the database unfortunately, it may be easier to create a new one.

In [21]:
new_tweets = pd.read_csv("http://www.stat.ucla.edu/~vlew/datasets/tweets_01-08-2021.csv")

pk_new_con = sqlite3.connect("new_tweets.db")
pk_new_cur = pk_new_con.cursor()
table_name = 'New_Tweets'

query = f'CREATE TABLE IF NOT EXISTS {table_name} (id, text, isRetweet, isDeleted, device, favorites, retweets, date, isFlagged)'
pk_new_cur.execute(query)
new_tweets.to_sql(table_name, pk_new_con, if_exists='replace', dtype={'id': 'INTEGER PRIMARY KEY AUTOINCREMENT'}, index=False)
  
pk_new_con.commit()
pk_new_cur.close()
pk_new_con.close()

### Always check

In [22]:
new_con = sqlite3.connect("new_tweets.db")
new_cur = new_con.cursor()
new_cur.execute('PRAGMA table_info(New_Tweets);')
table_info = new_cur.fetchall()

for column in table_info:
    print(column) # notnull FALSE, default None, PK)
    
new_cur.close()
new_con.close()

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'text', 'TEXT', 0, None, 0)
(2, 'isRetweet', 'TEXT', 0, None, 0)
(3, 'isDeleted', 'TEXT', 0, None, 0)
(4, 'device', 'TEXT', 0, None, 0)
(5, 'favorites', 'INTEGER', 0, None, 0)
(6, 'retweets', 'INTEGER', 0, None, 0)
(7, 'date', 'TEXT', 0, None, 0)
(8, 'isFlagged', 'TEXT', 0, None, 0)


## And back to pandas

We can also write our long SQL commands across multiple lines enclosed in triple quotes

In [23]:
query = '''
SELECT date, text, device
FROM New_Tweets
LIMIT 2;
'''

In [24]:
new_con = sqlite3.connect("new_tweets.db")
df = pd.read_sql_query(query, new_con)
new_con.close()
df[-2:]

,date,text,device
0,2009-05-04 18:54:25,Be sure to tune in and watch Donald Trump on L...,Twitter Web Client
1,2009-05-05 01:00:10,Donald Trump will be appearing on The View tom...,Twitter Web Client


# Writing Queries

<script>
    setBackgroundImage('Window1.jpg', 'black');
</script>

## `ORDER BY`

ORDER BY is SQL's version of sort

<https://www.w3schools.com/sql/sql_orderby.asp>

```
SELECT column1, column2, ...
FROM table_name
ORDER BY column1, column2, ... ASC|DESC;
```

In [25]:
my_con = sqlite3.connect("Chinook_Sqlite.sqlite")
command = '''
SELECT * 
FROM Employee 
ORDER BY Birthdate DESC;
'''
pd.read_sql_query(command, my_con)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
1,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
2,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
3,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
6,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
7,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com


## `WHERE`

Filter row selection with WHERE. (similar to using if as a boolean mask)

SQL uses single equal sign = for comparison

In [26]:
command = '''
SELECT * 
FROM Employee 
WHERE EmployeeId >= 6 AND Title = 'IT Staff'
ORDER BY BirthDate;
'''
pd.read_sql_query(command, my_con)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com


### `JOIN` and `LIMIT`

We can look at data across multiple tables using a `JOIN`

`LIMIT` acts like "head()", and limits the number of entries it returns

In [27]:
command = '''
SELECT * 
FROM Album
LIMIT 10;
'''
pd.read_sql_query(command, my_con)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,9,Plays Metallica By Four Cellos,7
9,10,Audioslave,8


In [28]:
command = '''
SELECT * 
FROM Artist
LIMIT 10;
'''
pd.read_sql_query(command, my_con)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
5,6,Antônio Carlos Jobim
6,7,Apocalyptica
7,8,Audioslave
8,9,BackBeat
9,10,Billy Cobham


`JOIN`

`INNER JOIN` is a specific type of join. It keeps only rows where the key exists in both tables. If one table is missing an entry that exists in the other table, the entry will not be returned.

When using a `JOIN`, specify the name of the table that is being joined and the columns used to match the rows. Columns are specified with dot notation. `TableName.ColumnName`

In [29]:
command = '''
SELECT * 
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
LIMIT 10;
'''
pd.read_sql_query(command, my_con)

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
5,6,Jagged Little Pill,4,Alanis Morissette
6,7,Facelift,5,Alice In Chains
7,8,Warner 25 Anos,6,Antônio Carlos Jobim
8,9,Plays Metallica By Four Cellos,7,Apocalyptica
9,10,Audioslave,8,Audioslave


In [30]:
# you can rename columns using `AS`
command = '''
SELECT Title AS "Album Title", Name AS "Artist Name"
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
LIMIT 10;
'''
pd.read_sql_query(command, my_con)

,Album Title,Artist Name
0,For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Accept
2,Restless and Wild,Accept
3,Let There Be Rock,AC/DC
4,Big Ones,Aerosmith
5,Jagged Little Pill,Alanis Morissette
6,Facelift,Alice In Chains
7,Warner 25 Anos,Antônio Carlos Jobim
8,Plays Metallica By Four Cellos,Apocalyptica
9,Audioslave,Audioslave


`GROUP BY` can be used to create groups to help calculate summary values

`COUNT()` is one function that can be used to calculate summary values. Other summary functions include `SUM()` and `AVG()`

In [31]:
command = '''
SELECT Artist.ArtistId, Name, COUNT(AlbumId) AS album_count,
  AVG(AlbumId) AS avg_id, SUM(AlbumID) as sum
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.ArtistId
LIMIT 10;
'''
pd.read_sql_query(command, my_con)

,ArtistId,Name,album_count,avg_id,sum
0,1,AC/DC,2,2.500000,5
1,2,Accept,2,2.500000,5
2,3,Aerosmith,1,5.000000,5
3,4,Alanis Morissette,1,6.000000,6
4,5,Alice In Chains,1,7.000000,7
5,6,Antônio Carlos Jobim,2,21.000000,42
6,7,Apocalyptica,1,9.000000,9
7,8,Audioslave,3,97.333333,292
8,9,BackBeat,1,12.000000,12
9,10,Billy Cobham,1,13.000000,13


In [32]:
# for comparison with previous table
command = '''
SELECT * 
FROM Album
ORDER BY ArtistId
LIMIT 15;
'''

pd.read_sql_query(command, my_con)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,4,Let There Be Rock,1
2,2,Balls to the Wall,2
3,3,Restless and Wild,2
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,34,Chill: Brazil (Disc 2),6
9,9,Plays Metallica By Four Cellos,7


In [33]:
# Conditionals on the Group By must be done with 'HAVING'
command = '''
SELECT Artist.ArtistId, Name, COUNT(AlbumId) AS album_count
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.ArtistId
HAVING album_count > 8;
'''
pd.read_sql_query(command, my_con)

,ArtistId,Name,album_count
0,22,Led Zeppelin,14
1,50,Metallica,10
2,58,Deep Purple,11
3,90,Iron Maiden,21
4,150,U2,10


In [34]:
command = '''
SELECT ArtistId, ArtistId * 2 AS "magic number", Name
From Artist 
LIMIT 10;
'''
pd.read_sql_query(command,  my_con)

,ArtistId,magic number,Name
0,1,2,AC/DC
1,2,4,Accept
2,3,6,Aerosmith
3,4,8,Alanis Morissette
4,5,10,Alice In Chains
5,6,12,Antônio Carlos Jobim
6,7,14,Apocalyptica
7,8,16,Audioslave
8,9,18,BackBeat
9,10,20,Billy Cobham


## Table previews

In [35]:
command = '''
SELECT * 
FROM Album
LIMIT 5;
'''
pd.read_sql_query(command,  my_con)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [36]:
command = '''
SELECT * 
FROM Artist
LIMIT 5;
'''
pd.read_sql_query(command,  my_con)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [37]:
command = '''
SELECT * 
FROM Invoice
LIMIT 5;
'''
pd.read_sql_query(command,  my_con)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [38]:
command = '''
SELECT * 
FROM InvoiceLine
LIMIT 7;
'''
pd.read_sql_query(command,  my_con)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
5,6,2,12,0.99,1
6,7,3,16,0.99,1


In [39]:
command = '''
SELECT * 
FROM Track
LIMIT 6;
'''
pd.read_sql_query(command,  my_con)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
5,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99


In [40]:
command = '''
SELECT * 
FROM Customer
LIMIT 5;
'''
pd.read_sql_query(command,  my_con)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


## table aliases

Provide a query showing the Invoices of customers who are from Brazil. The resultant table should show the customer's full name, Invoice ID, Date of the invoice and billing country.

In [41]:
command = '''
SELECT c.FirstName, c.lastname, 
    i.invoiceid, i.invoicedate, i.billingcountry    -- selects the desired columns
FROM customer AS c                     -- provide an alias to the table, so we dont have to type the full name out
    JOIN invoice AS i
    ON c.customerid = i.customerid     -- this is how the tables are linked
WHERE c.country = 'Brazil'
LIMIT 20;                               -- limits how many rows we get back
'''
pd.read_sql_query(command,  my_con)

,FirstName,LastName,InvoiceId,InvoiceDate,BillingCountry
0,Luís,Gonçalves,98,2010-03-11 00:00:00,Brazil
1,Luís,Gonçalves,121,2010-06-13 00:00:00,Brazil
2,Luís,Gonçalves,143,2010-09-15 00:00:00,Brazil
3,Luís,Gonçalves,195,2011-05-06 00:00:00,Brazil
4,Luís,Gonçalves,316,2012-10-27 00:00:00,Brazil
5,Luís,Gonçalves,327,2012-12-07 00:00:00,Brazil
6,Luís,Gonçalves,382,2013-08-07 00:00:00,Brazil
7,Eduardo,Martins,25,2009-04-09 00:00:00,Brazil
8,Eduardo,Martins,154,2010-11-14 00:00:00,Brazil
9,Eduardo,Martins,177,2011-02-16 00:00:00,Brazil


## `DISTINCT` 

Provide a query showing a unique list of billing countries from the Invoice table.

In [42]:
command = '''
SELECT DISTINCT billingcountry 
FROM invoice;
'''
pd.read_sql_query(command,  my_con)

,BillingCountry
0,Germany
1,Norway
2,Belgium
3,Canada
4,USA
5,France
6,Ireland
7,United Kingdom
8,Australia
9,Chile


## Joining three (or more) tables

The following query shows the invoices associated with each sales agent.

The invoice table has no information about employee. But each invoice has a customer and each customer has a support rep (employee). We connect the invoice table with the employee table by connecting them through the customer table

In [43]:
command = '''
SELECT e.firstname, e.lastname,   -- employee first and last name
       i.*   -- all columns from invoice table 

FROM invoice AS i
    JOIN customer AS c
    ON c.customerid = i.customerid

    JOIN employee AS e
    ON e.employeeid = c.supportrepid
    
ORDER BY e.employeeid;
'''
pd.read_sql_query(command,  my_con)

,FirstName,LastName,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,Jane,Peacock,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
1,Jane,Peacock,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
2,Jane,Peacock,9,42,2009-02-02 00:00:00,"9, Place Louis Barthou",Bordeaux,None,France,33000,3.96
3,Jane,Peacock,10,46,2009-02-03 00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94
4,Jane,Peacock,11,52,2009-02-06 00:00:00,202 Hoxton Street,London,None,United Kingdom,N1 5LH,8.91
...,...,...,...,...,...,...,...,...,...,...,...
407,Steve,Johnson,398,41,2013-10-21 00:00:00,"11, Place Bellecour",Lyon,None,France,69002,0.99
408,Steve,Johnson,402,50,2013-11-05 00:00:00,C/ San Bernardo 85,Madrid,None,Spain,28015,5.94
409,Steve,Johnson,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86
410,Steve,Johnson,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98


Following query shows the Invoice Total, Customer name, Country and Sale Agent name for all invoices and customers.

In [44]:
command = '''
SELECT i.InvoiceId, i.total,
       e.firstname AS 'employee first', 
       e.lastname AS 'employee last', 
       c.firstname AS 'customer first', 
       c.lastname AS 'customer last', 
       c.country
FROM employee AS e
        JOIN customer AS c 
        ON e.employeeid = c.supportrepid
        JOIN invoice AS i 
        ON c.customerid = i.customerid;
'''
pd.read_sql_query(command,  my_con)

,InvoiceId,Total,employee first,employee last,customer first,customer last,Country
0,1,1.98,Steve,Johnson,Leonie,Köhler,Germany
1,2,3.96,Margaret,Park,Bjørn,Hansen,Norway
2,3,5.94,Margaret,Park,Daan,Peeters,Belgium
3,4,8.91,Steve,Johnson,Mark,Philips,Canada
4,5,13.86,Margaret,Park,John,Gordon,USA
...,...,...,...,...,...,...,...
407,408,3.96,Steve,Johnson,Victor,Stevens,USA
408,409,5.94,Jane,Peacock,Robert,Brown,Canada
409,410,8.91,Margaret,Park,Madalena,Sampaio,Portugal
410,411,13.86,Jane,Peacock,Terhi,Hämäläinen,Finland


How many Invoices were there in 2011? What are the total sales for that year?

In [45]:
command = '''
SELECT invoiceId, InvoiceDate, total
FROM invoice as i
WHERE i.invoicedate BETWEEN datetime('2011-01-01') 
AND datetime('2011-12-31');
'''
pd.read_sql_query(command,  my_con)  # result has 83 rows

,InvoiceId,InvoiceDate,Total
0,167,2011-01-02 00:00:00,0.99
1,168,2011-01-15 00:00:00,1.98
2,169,2011-01-15 00:00:00,1.98
3,170,2011-01-16 00:00:00,3.96
4,171,2011-01-17 00:00:00,5.94
...,...,...,...
78,245,2011-12-22 00:00:00,1.98
79,246,2011-12-22 00:00:00,1.98
80,247,2011-12-23 00:00:00,3.96
81,248,2011-12-24 00:00:00,5.94


In [46]:
command = '''
SELECT count(i.invoiceId) as 'count',
    sum(i.total) as 'sum'
FROM invoice as i
WHERE i.invoicedate BETWEEN datetime('2011-01-01')
AND datetime('2011-12-31');
'''
pd.read_sql_query(command,  my_con)

,count,sum
0,83,469.58


Count how many orders were made on each day

In [47]:
command = '''
SELECT i.InvoiceDate, count(i.invoiceId) as 'count'
FROM invoice as i
WHERE i.invoicedate BETWEEN datetime('2011-01-01') AND datetime('2011-12-31')
GROUP BY i.invoiceDate;
'''
pd.read_sql_query(command,  my_con)

,InvoiceDate,count
0,2011-01-02 00:00:00,1
1,2011-01-15 00:00:00,2
2,2011-01-16 00:00:00,1
3,2011-01-17 00:00:00,1
4,2011-01-20 00:00:00,1
...,...,...
66,2011-12-09 00:00:00,1
67,2011-12-22 00:00:00,2
68,2011-12-23 00:00:00,1
69,2011-12-24 00:00:00,1


Looking at the InvoiceLine table, provide a query that COUNTs the number of line items for each Invoice.

In [48]:
command = '''
SELECT *
FROM invoiceline
LIMIT 10;
'''
pd.read_sql_query(command,  my_con)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
5,6,2,12,0.99,1
6,7,3,16,0.99,1
7,8,3,20,0.99,1
8,9,3,24,0.99,1
9,10,3,28,0.99,1


In [49]:
command = '''
SELECT invoiceid, count(invoicelineid) AS 'Count'
FROM invoiceline
GROUP BY invoiceid
ORDER BY Count DESC;
'''
pd.read_sql_query(command,  my_con)

,InvoiceId,Count
0,5,14
1,12,14
2,19,14
3,26,14
4,33,14
...,...,...
407,384,1
408,391,1
409,398,1
410,405,1


Find the invoice with the maximum number of Invoiceline IDs most elegant please

CTE Common Table Expression - allows you to query tables that you created as intermediate steps

In [50]:
command = '''
WITH InvoiceCounts (id, count) 
AS 
(  -- an intermediate table that aggregates the invoicelineIDs 
   -- pretty much the exact same table we generated in previous step
    SELECT invoiceid, count(invoicelineid) AS 'Count'
    FROM invoiceline
    GROUP BY invoiceid
    ORDER BY Count DESC
)

SELECT MIN(count) as Min, MAX(count) as Max
FROM InvoiceCounts;
'''
pd.read_sql_query(command,  my_con)

,Min,Max
0,1,14


In [51]:
command = '''
WITH InvoiceCounts (id, count) 
AS 
(  -- an intermediate table that aggregates the invoicelineIDs 
   -- pretty much the exact same table we generated in previous step
    SELECT invoiceid, count(invoicelineid) AS 'Count'
    FROM invoiceline
    GROUP BY invoiceid
    ORDER BY Count DESC
)

SELECT count, COUNT(id) as "HowMany"
FROM InvoiceCounts
GROUP BY count;
'''
pd.read_sql_query(command, my_con)

,count,HowMany
0,1,59
1,2,117
2,4,59
3,6,59
4,9,59
5,14,59


Provide a query that includes the purchased track name AND artist name with each invoice line item.


In [52]:
command = '''
SELECT i.*, 
    t.name AS 'track', 
    ar.name AS 'artist'
FROM invoiceline AS i
        JOIN track AS t 
            ON i.trackid = t.trackid     -- i links to t
        JOIN album AS al 
            ON t.albumid = al.albumid    -- t links to al
        JOIN artist AS ar 
            ON al.artistid = ar.artistid;  -- al links to ar
'''
pd.read_sql_query(command, my_con)
my_con.close()

## More queries multiple tables

In [53]:
# Use the inspector to get table names
# Save the table names to a list: table_names
my_con = sqlite3.connect("employee.sqlite")
res = my_con.execute("SELECT name FROM sqlite_master")
res.fetchall()

[('emp',), ('dept',)]

In [54]:
command = '''
SELECT * FROM emp
'''
pd.read_sql_query(command, my_con)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,2005-12-17,800,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,2006-02-20,1600,300.0,30
2,7521,WARD,SALESMAN,7698.0,2006-02-22,1250,500.0,30
3,7566,JONES,MANAGER,7839.0,2006-04-02,2975,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,2006-09-28,1250,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,2006-05-01,2850,NaN,30
6,7782,CLARK,MANAGER,7839.0,2006-06-09,2450,NaN,10
7,7788,SCOTT,ANALYST,7566.0,2007-12-09,3000,NaN,20
8,7839,KING,PRESIDENT,NaN,2006-11-17,5000,NaN,10
9,7844,TURNER,SALESMAN,7698.0,2006-09-08,1500,0.0,30


In [55]:
command = '''
SELECT * FROM dept
'''
pd.read_sql_query(command, my_con)

,DEPTNO,DNAME,LOC
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON


### Concatenating Column Values

You want to return values in multiple columns as one column. For example, you would like to produce this result set from a query against the EMP table:

```
CLARK WORKS AS A MANAGER
KING WORKS AS A PRESIDENT
MILLER WORKS AS A CLERK
```

Use the double vertical bar as the concatenation operator

In [56]:
command = '''
SELECT ENAME||' WORKS AS A '||JOB AS msg
FROM emp
WHERE deptno=10
'''
pd.read_sql_query(command, my_con)

,msg
0,CLARK WORKS AS A MANAGER
1,KING WORKS AS A PRESIDENT
2,MILLER WORKS AS A CLERK


### Using Conditional Logic in a SELECT Statement

You want to perform IF-ELSE operations on values in your SELECT statement. For example, you would like to produce a result set such that if an employee is paid \\$2,000 or less, a message of "lower wage" is returned; if an employee is paid \\$4,000 or more, a message of "higher wage" is returned; and if they make somewhere in between, then "middle" is returned.

Use the CASE expression to perform conditional logic directly in your SELECT statement

In [57]:
command = '''
SELECT ENAME, sal,
        CASE WHEN sal <= 2000 THEN 'lower wage'
             WHEN sal >= 4000 THEN 'higher wage'
             ELSE 'middle'
        END
        AS status
FROM emp
'''
pd.read_sql_query(command, my_con)

,ENAME,SAL,status
0,SMITH,800,lower wage
1,ALLEN,1600,lower wage
2,WARD,1250,lower wage
3,JONES,2975,middle
4,MARTIN,1250,lower wage
5,BLAKE,2850,middle
6,CLARK,2450,middle
7,SCOTT,3000,middle
8,KING,5000,higher wage
9,TURNER,1500,lower wage


### Returning n Random Records from a Table

You want to return a specific number of random records from a table. 

Use the built-in RANDOM function in conjunction with LIMIT and ORDER BY

In [58]:
command = '''
SELECT ENAME,JOB
FROM emp
ORDER BY RANDOM()
LIMIT 5
'''
pd.read_sql_query(command, my_con)

,ENAME,JOB
0,MILLER,CLERK
1,CLARK,MANAGER
2,SMITH,CLERK
3,MARTIN,SALESMAN
4,ADAMS,CLERK


### Finding Null Values

You want to find all rows that are null for a particular column.

To determine whether a value is null, you must use IS NULL

In [59]:
command = '''
SELECT *
FROM emp
WHERE comm IS NULL
'''
pd.read_sql_query(command, my_con)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,2005-12-17,800,None,20
1,7566,JONES,MANAGER,7839.0,2006-04-02,2975,None,20
2,7698,BLAKE,MANAGER,7839.0,2006-05-01,2850,None,30
3,7782,CLARK,MANAGER,7839.0,2006-06-09,2450,None,10
4,7788,SCOTT,ANALYST,7566.0,2007-12-09,3000,None,20
5,7839,KING,PRESIDENT,NaN,2006-11-17,5000,None,10
6,7876,ADAMS,CLERK,7788.0,2008-01-12,1100,None,20
7,7900,JAMES,CLERK,7698.0,2006-12-03,950,None,30
8,7902,FORD,ANALYST,7566.0,2006-12-03,3000,None,20
9,7934,MILLER,CLERK,7782.0,2007-01-23,1300,None,10


### Transforming Nulls into Real Values

You have rows that contain nulls and would like to return non-null values in place of those nulls.

Use the function COALESCE to substitute real values for nulls:

In [60]:
command = '''
SELECT ENAME, COMM, COALESCE(COMM, 0) AS COMMISION
FROM emp
'''
pd.read_sql_query(command, my_con)

,ENAME,COMM,COMMISION
0,SMITH,NaN,0
1,ALLEN,300.0,300
2,WARD,500.0,500
3,JONES,NaN,0
4,MARTIN,1400.0,1400
5,BLAKE,NaN,0
6,CLARK,NaN,0
7,SCOTT,NaN,0
8,KING,NaN,0
9,TURNER,0.0,0


### Searching for Patterns

You want to return rows that match a particular substring or pattern.

Use the LIKE operator in conjunction with the SQL wildcard operator (%)

When used in a LIKE pattern-match operation, the percent (%) operator matches any sequence of characters. Most SQL implementations also provide the underscore ("\_") operator to match a single character. By enclosing the search pattern "I" with \% operators, any string that contains an "I" (at any position) will be returned. If you do not enclose the search pattern with %, then where you place the operator will affect the results of the query. 

For example, to find job titles that end in "ER," use "%ER". If the requirement is to search for all job titles beginning with "ER," use "ER%"

In [61]:
command = '''
SELECT ename, job
FROM emp
WHERE deptno IN (10,20)
   AND (ename LIKE '%I%' OR job LIKE '%ER') --# employee name contains an I or job ends with 'er"
'''
pd.read_sql_query(command, my_con)
my_con.close()

<h1> Statistics 21 <br/> Have a Good Night! </h1>

<script>
    setBackgroundImage('Window1.jpg', 'black');
</script>